# Week 4 Extending SQL by PostgreSQL

## Ways that SQL could be extended

* To accurately express the “business logic“ of withdrawing money, we need facilities like conditional controls

## A Stored Procedure Approach

## SQL/PSM


## PostgreSQL

A PostgreSQL-specific language integrating features of:
* procedural programming (PL)
* SQL programming

### User-defined Data Types
PostgreSQL also provides mechanisms to define new types
* basic types: CREATE DOMAIN
* tuple types: CREATE TYPE

> Example:
> 
> Create Domain UnswCourseCode as text
> 
> check (value ~ '[A-Z]{4}[0-9]{4}' );

### PostgreSQL: SQL Functions

```
CREATE OR REPLACE FUNCTION
funcName(arg1type, arg2type, ....)
RETURNS rettype
AS $$
SQL statements
$$ LANGUAGE sql;
```

Example:
```
-- max price of specified beer
create or replace function
    maxPrice(text) returns float
as $$
    select max(price) from Sells where beer = $1;
$$ language sql;
```
using positional notation ($1, $2, ...)

### PL/pgSQL Function Parameters

```
    CREATE OR REPLACE FUNCTION
        funcName(param1, param2, ....)
    RETURNS rettype
    AS $$
    DECLARE
        variable declarations
    BEGIN
        code for function
    END;
    $$ LANGUAGE plpgsql;
```

> Note: the entire function body is a single SQL string

Example:
```
    CREATE OR REPLACE FUNCTION
        add(x text, y text) RETURNS text
    AS $$
    DECLARE
        result text; -- local variable
    BEGIN
        result := x||''''||y;
        return result;
    END;
    $$ LANGUAGE plpgsql;
```

### Using PL/pgSQL Functions
* As part of a SELECT statement

```
select myFunction ( arg1 , arg2 );
select * from myTableFunction ( arg1 , arg2 );
```

* As part of the execution of another PLpgSQL function
```
PERFORM myVoidFunction ( arg1 , arg2 );
result := myOtherFunction ( arg1 );

```
* Automatically, via an insert/delete/update trigger

```
create trigger T before update on R
for each row execute procedure myCheck ();
```

## Declaring Data Types

```
account Accounts%ROWTYPE ;
```

```
account.branchName%TYPE
```

## Control Structures in PL/pgSQL

### Assignment 
variable := expression;

Example:
```
    tax := subtotal * 0.06;
    my_record.user_id := 20;
```

### Conditionals
* IF ... THEN
* IF ... THEN ... ELSE
* IF ... THEN ... ELSIF ... THEN ... ELSE
Example:

```
IF v_user_id > 0 THEN
UPDATE users SET email = v_email WHERE user_id =
v_user_id; END IF;
```


```
create function
        withdraw(acctNum text, amount integer) returns text as $$
declare bal integer;
begin
        select balance into bal
        from Accounts
        where acctNo = acctNum;
        if (bal < amount) then
                return 'Insufficient Funds';
        else
                update Accounts
                set balance = balance - amount
                where acctNo = acctNum;
                select balance into bal
                from Accounts where acctNo = acctNum;
                return 'New Balance: ' || bal;
        end if;
end;
$$ language plpgsql;
```

## Control Structures

```
LOOP
    Statement
END LOOP ;
```
```
FOR int_var IN low .. high LOOP
    Statement
END LOOP 
```

## SELECT ... INTO

Capture query result to assign to variable

## Exceptions

To catch the error 

```
    BEGIN
        Statements ...
    EXCEPTION
        WHEN Exceptions1 THEN
            StatementsForHandler1
        WHEN Exceptions2 THEN
            StatementsForHandler2
        ...
    END ;
```

```
    -- table T contains one tuple ( ' Tom ' , ' Jones ')
    DECLARE
        x INTEGER := 3;
    BEGIN
        UPDATE T SET firstname = 'Joe' WHERE lastname = ' Jones ';
        -- table T now contains ( ' Joe ' , ' Jones ')
        x := x + 1;
        y := x / 0;
    EXCEPTION
        WHEN division_by_zero THEN
        -- update on T is rolled back to ( 'Tom' , ' Jones ')
        RAISE NOTICE ' Caught division_by_zero ';
        RETURN x ;
        -- value returned is 4
    END ;
```

The RAISE operator generates server log entries, e.g.
* RAISE DEBUG 'Simple message';
* RAISE NOTICE ' User = % ' , user_id ;
* RAISE EXCEPTION ' Fatal : value was % ' , value ;
> DEBUG, LOG, INFO, NOTICE, WARNING, and EXCEPTION

## Cursors
A cursor is an object that retrieves rows from a result table

RECORD variable or Table%ROWTYPE variable

* Save network bandwidth and time. We don’t need to wait for whole result set to be retrieved/ processed.
* Since the cursor already stores the value of a row, other database processes can continue to update or delete other rows on the table,
* You can return a cursor in a pl/pgsql function.

```
    Create Function totalSalary() Returns real As $$
    Declare
        employee RECORD;
        totalSalary REAL:=0;
    Begin
        FOR employee IN SELECT * FROM Employees
    Loop
        totalSalary:=totalSalary+employee.salary ;
    End Loop;
    Return total;
    End ; $$ Language plpgsql;
```

## Opening and Closing Cursors


## Fetching Cursors

```
DECLARE
    my_cursor CURSOR FOR
        SELECT column1, column2 FROM my_table;
    row_record my_table%ROWTYPE;
BEGIN
    OPEN my_cursor;
    LOOP
        FETCH my_cursor INTO row_record;
        EXIT WHEN NOT FOUND;
        -- Process row_record here
    END LOOP;
    CLOSE my_cursor;
END;
```

## Database Triggers

* an event activates the trigger
* on activation, the trigger checks a condition
* if the condition holds, a procedure will excecuted

Eg:

```
    CREATE TRIGGER TriggerName
    AFTER/BEFORE Event1 [OR Event2 ...]
    ON TableName
    FOR EACH ROW/STATEMENT
    EXECUTE PROCEDURE FunctionName(args...);
```

## Types of Triggers 

* Row level triggers
* Statement-level triggers

## Old and New


### Insert

### Update

### Delete
